In [ ]:
import pandas as pd
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt 
import tushare as ts
import akshare as ak
import sys,os


sys.stdout.encoding = 'utf_8_sig'
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

In [ ]:
stock_list = pd.read_csv('stock_list.csv', encoding='utf_8_sig')
stock_list_CSI300 = ak.index_stock_cons_csindex(symbol="000300") #导入沪深300成分股的数据

In [ ]:
#跟stock_list中的代码对比，显示没有的数据
files = os.listdir('Data_k')
files.sort()
files[0][files[0].find('_')+1:files[0].find('_')+10]
Codes = []
for file in files:
    Codes.append(file[file.find('_')+1:file.find('_')+10])
stock_list = pd.read_csv('stock_list.csv', encoding='utf_8_sig')
stocks_none = stock_list[~stock_list['Code'].isin(Codes)]
#stocks_none = [x for x in stock_list['Code'] if x not in Codes]
print(stocks_none)

#生成后复权的收盘价
# stock1 = pd.read_csv("Data_k/"+files[300], encoding='utf_8_sig')
# stock1.head()

#导出财务数据，PE PB等


In [ ]:
files[0:6]

# 计算除复权价格

In [ ]:
stock_k = pd.DataFrame()
for file in files[0:4]:
    df = pd.read_csv('Data_k/'+file, encoding='utf_8_sig')
    if len(df) == 0:
        print(file, "数据为空")
        continue
    # 计算涨跌幅
    df['chg'] = df['close'].shift(1) / df['pre_close'] - 1
    df.iloc[0,11] = 0 #第11列为'chg'
    # =计算复权价: 计算所有因当当中用到的价格, 都使用复权价
    df['adj_factor'] = (1 + df['chg']).cumprod()

    label = 'after'  # 选择复权模式的标签
    # 计算后复权价格
    if label == 'after':
        df['adj_after_close'] = df['adj_factor'] * df['close']
    # 计算前复权价格
    if label == 'pre':
        df['adj_pre_close'] = df['adj_factor'] * (
                df.iloc[-1]['close'] / df.iloc[-1]['adj_factor'])
    stock_k = pd.concat([stock_k, df], axis=0)

stock_k['trade_date'] = pd.to_datetime(stock_k['trade_date'].astype("str"))
stock_k.set_index("trade_date",inplace=True,drop=True)
# # 计算其他价格的复权
# df['开盘价%s复权' % label] = df['开盘价'] / df['收盘价'] * df['收盘价%s复权' % label]
# df['最高价%s复权' % label] = df['最高价'] / df['收盘价'] * df['收盘价%s复权' % label]
# df['最低价%s复权' % label] = df['最低价'] / df['收盘价'] * df['收盘价%s复权' % label]

In [ ]:
def get_data_k(syboml):
    df = stock_k[stock_k["ts_code"]==syboml][["open","high","low","adj_after_close","vol"]]
    df['openinterest'] = 0
    df.rename(columns={"adj_after_close": "close", "vol": "volume"}, inplace=True)
    return df

get_data_k('300076.SZ').head()

In [ ]:
# 通过AKshare获取A股所有股票代码
stock_info_a_code_name_df = ak.stock_info_a_code_name()

def AppendExchange(code):
    if code[0] == '0':
        return code+'.SZ'
    elif code[0] == '3':
        return code+'.SZ'
    elif code[0] == '4':
        return code+'.BJ'
    elif code[0] == '5':
        return code+'.SZ'
    elif code[0] == '3':
        return code+'.SZ'
    elif code[0] == '6':
        return code+'.SH'
    elif code[0] == '8':
        return code+'.BJ'
    elif code[0] == '9':
        return code+'.BJ'
    else:
        print(code)
        return code

stock_info_a_code_name_df['Code'] = stock_info_a_code_name_df['code'].apply(lambda x: AppendExchange(x))
stock_info_a_code_name_df.to_csv('stock_list.csv', index=False,encoding='utf_8_sig')

In [ ]:
#沪深300成分股
index_stock_cons_csindex_df = ak.index_stock_cons_csindex(symbol="000300")
#index_stock_cons_csindex_df.to_csv('Data/'+'stock_list_CSI300.csv', index=False,encoding='utf_8_sig')
index_stock_cons_csindex_df.head()

# 获取A股K线数据，并存成csv文件

In [ ]:
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()

# 获取 A 股所有上市公司的代码和名称
stock_list = pd.read_csv('stock_list.csv', encoding='utf_8_sig')

start_date = "20050101"
end_date = "20241231"
#tushare 存储股票数据进入CSV
for index, row in stock_list.iterrows():
    stock_code = row['Code']
    stock_name = row['name']
    # 组合文件名，例如：贵州茅台 600519.SH.csv
    file_name = f"{stock_name}_{stock_code}.csv"
    #f = open("Data/results.csv","a",encoding='utf_8_sig')

    try:
        # 获取股票日线行情数据，可根据实际需求换用其他合适的数据接口
        stock_data = pro.daily(ts_code=stock_code, start_date=start_date, end_date=end_date)
        stock_data = stock_data.iloc[::-1]
        # 将数据保存为 CSV 文件
        stock_data.to_csv('Data/'+file_name, index=False,encoding='utf_8_sig')
        #print(f"{index}-{stock_name}（{stock_code}）的数据已成功保存到 {file_name}",file=f)
        print(f"{index}-{stock_name}（{stock_code}）的数据已成功保存到 {file_name}")
    except Exception as e:
        #print(f"获取 {index}-{stock_name}（{stock_code}）的数据时出现错误：{str(e)}",file=f)
        print(f"获取 {index}-{stock_name}（{stock_code}）的数据时出现错误：{str(e)}")

    #f.close()


In [ ]:
stock_list = pd.read_csv('stock_list.csv', encoding='utf_8_sig')
stock_list.head()

In [ ]:
a = '30076.sz'
a[a.find('_')+1:a.find('.')]
ak.stock_financial_abstract_ths(symbol='002141',indicator="按报告期")
a = ['000506,']

In [ ]:
stock_list = pd.read_csv('stock_11.csv', encoding='utf_8_sig')
stock_list.head()


# 获取A股财务数据，并存成csv文件

In [ ]:
stock_list = pd.read_csv('stock_list.csv', encoding='utf_8_sig')

for index, row in stock_list.iterrows():
    stock_code = row['Code']
    stock_name = row['name']
    # 组合文件名，例如：贵州茅台 600519.SH.csv
    file_name = f"{stock_name}_{stock_code}.csv"
    f = open("results_financial_abstract.csv","a",encoding='utf_8_sig')

    try:
        # 获取财务摘要数据
        stock_data = ak.stock_financial_abstract_ths(symbol=stock_code[:stock_code.find('.')],indicator="按报告期")
        # 将数据保存为 CSV 文件
        stock_data.to_csv('Data_financial_abstract/'+file_name, index=False,encoding='utf_8_sig')
        print(f"{index}-{stock_name}（{stock_code}）的数据已成功保存到 {file_name}",file=f)
        print(f"{index}-{stock_name}（{stock_code}）的数据已成功保存到 {file_name}")
    except Exception as e:
        print(f"获取 {index}-{stock_name}（{stock_code}）的数据时出现错误：{str(e)}",file=f)
        print(f"获取 {index}-{stock_name}（{stock_code}）的数据时出现错误：{str(e)}")

    f.close()

# tushare数据接口并转换为backtrader能识别的格式

In [ ]:
#tushare Pro版本
ts.set_token('5d3c09f16bc3ad409898ff6cfc10aadeeae28ba23aa240c047dd125c')
pro = ts.pro_api()

start_date = '20170101'
end_date = '20211201'
code = '600519.SH'

def get_data(code = code,start_date = start_date,end_date = end_date):
    df = pro.daily(ts_code = code,start_date = start_date,end_date = end_date)
    df = df.iloc[::-1]
    df.index = pd.to_datetime(df.trade_date)
    df['openinterest'] = 0
    df = df[['open','high','low','close','vol','openinterest']]
    df.rename(columns={'vol': 'volume'}, inplace=True)
    return df

stock_df = get_data()
data = bt.feeds.PandasData(
    dataname=stock_df,
    fromdate=pd.to_datetime(start_date),
    todate=pd.to_datetime(end_date),
)

In [ ]:
# tushare 老版本的方式
code = '600519'
start_date = '2017-01-01'
end_date = '2018-05-01'

def get_data(code = code,start_date = start_date,end_date = end_date):
    df = ts.get_k_data(code,start = start_date,end = end_date)
    df.index = pd.to_datetime(df.date)
    df['openinterest'] = 0
    # 对df的数据列进行一个整合
    df = df[['open','high','low','close','volume','openinterest']]
    print(df.head())
    print(df.tail())
    print(len(df))
    return df

stock_df = get_data()

data = bt.feeds.PandasData(
    dataname=stock_df,
    fromdate=pd.to_datetime(start_date),
    todate=pd.to_datetime(end_date)
)

In [ ]:
stock = qs.utils.download_returns('TSLA')
sp500 = qs.utils.download_returns('SPY')
qs.reports.metrics(stock, benchmark=sp500)
#qs.reports.basic(stock, benchmark=sp500)
#qs.reports.full(stock, benchmark=sp500)
#qs.plots.snapshot(stock,show=True)
#qs.reports.html(stock, benchmark=sp500,output='report.html')

# backtrader接口模版

In [ ]:
#原始事件策略模板，请基于此改写
class my_strategy_date(bt.Strategy):
    #全局设定交易策略的参数

    def __init__(self):
        # 初始化交易指令、买卖价格和手续费
        self.order = None

    def next(self):
        # 检查是否持仓 
        #print(str(self.datetime.date(0)))
        if str(self.datetime.date(0)) in sell_date.keys(): 
            s_list=sell_date[str(self.datetime.date(0))]
            for i in s_list:
                self.order_target_percent(target=0,data=i)
                
        if str(self.datetime.date(0)) in buy_date.keys(): 
            s_list=buy_date[str(self.datetime.date(0))]
            for i in s_list:
                self.order_target_percent(target=0.9/len(s_list),data=i)
                

            
    def log(self, txt, dt=None):
        ''' 输出日志'''
        dt = dt or self.datas[0].datetime.date(0) # 拿现在的日期
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(f"""买入{order.data._name}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            elif order.issell():
                self.log(f"""卖出{order.data._name}, 成交量{order.executed.size}，成交价{order.executed.price:.2f}""")
            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

In [ ]:
"""
股票数据采集器
使用akshare获取股票历史数据的完整解决方案

功能特点:
1. 单个股票数据获取
2. 批量股票数据获取
3. 增量更新数据（避免重复下载）
4. 自动创建目录和文件管理
5. 异常处理和进度提示

作者: B站添爸学python
日期: 2025年5月
"""

import akshare as ak
import pandas as pd
import os
from datetime import datetime, timedelta
import time


class StockDataCollector:
    """股票数据采集器类"""
    
    def __init__(self, data_dir="股票数据"):
        self.data_dir = data_dir
        self._ensure_dir_exists()
    
    def _ensure_dir_exists(self):
        """确保数据目录存在"""
        if not os.path.exists(self.data_dir):
            os.makedirs(self.data_dir)
            print(f"已创建数据目录: {self.data_dir}")
    
    def get_single_stock(self, stock_code, stock_name=None, 
                        start_date="20230101", end_date=None):
        """
        获取单个股票数据
        
        参数:
        stock_code: 股票代码 (如"601318")
        stock_name: 股票名称 (如"中国平安")，可选
        start_date: 开始日期 (格式: "20230101")
        end_date: 结束日期 (格式: "20231231")，默认为今天
        
        返回:
        DataFrame: 股票数据
        """
        if end_date is None:
            end_date = datetime.now().strftime("%Y%m%d")
        
        if stock_name is None:
            stock_name = f"股票_{stock_code}"
        
        print(f"正在获取 {stock_name}({stock_code}) 数据...")
        print(f"日期范围: {start_date} 到 {end_date}")
        
        try:
            # 获取股票历史日K数据
            stock_data = ak.stock_zh_a_hist(
                symbol=stock_code, 
                period="daily", 
                start_date=start_date, 
                end_date=end_date,
                adjust=""  # 不复权
            )
            
            if stock_data.empty:
                print(f"警告: {stock_name} 没有获取到数据")
                return stock_data
            
            # 保存到Excel文件
            file_name = f"{self.data_dir}/{stock_name}_{stock_code}_日K数据.xlsx"
            stock_data.to_excel(file_name, index=False)
            
            print(f"✓ 成功保存 {stock_name} 数据到 {file_name}")
            print(f"  数据条数: {len(stock_data)}")
            print(f"  日期范围: {stock_data['日期'].min()} 到 {stock_data['日期'].max()}")
            
            return stock_data
            
        except Exception as e:
            print(f"✗ 获取 {stock_name} 数据失败: {e}")
            return pd.DataFrame()
    
    def batch_get_stocks(self, stock_list, stock_names=None, 
                        start_date="20230101", end_date=None):
        """
        批量获取多个股票数据
        
        参数:
        stock_list: 股票代码列表 (如["600519", "601318"])
        stock_names: 股票名称列表 (如["贵州茅台", "中国平安"])，可选
        start_date: 开始日期
        end_date: 结束日期，默认为今天
        
        返回:
        DataFrame: 汇总的股票数据
        """
        if end_date is None:
            end_date = datetime.now().strftime("%Y%m%d")
        
        if stock_names is None:
            stock_names = [f"股票_{code}" for code in stock_list]
        
        if len(stock_list) != len(stock_names):
            raise ValueError("股票代码列表和股票名称列表长度不匹配")
        
        print(f"开始批量获取 {len(stock_list)} 只股票数据...")
        print("=" * 50)
        
        all_data = pd.DataFrame()
        success_count = 0
        
        for i, stock_code in enumerate(stock_list):
            stock_name = stock_names[i]
            print(f"[{i+1}/{len(stock_list)}] 处理 {stock_name}({stock_code})")
            
            try:
                # 获取股票历史数据
                stock_data = ak.stock_zh_a_hist(
                    symbol=stock_code, 
                    period="daily", 
                    start_date=start_date, 
                    end_date=end_date,
                    adjust=""
                )
                
                if not stock_data.empty:
                    # 保存单个股票文件
                    file_name = f"{self.data_dir}/{stock_name}_{stock_code}_日K数据.xlsx"
                    stock_data.to_excel(file_name, index=False)
                    
                    # 添加股票标识信息
                    stock_data["股票代码"] = stock_code
                    stock_data["股票名称"] = stock_name
                    all_data = pd.concat([all_data, stock_data], ignore_index=True)
                    
                    success_count += 1
                    print(f"  ✓ 成功获取 {len(stock_data)} 条数据")
                else:
                    print(f"  ⚠ 没有获取到数据")
                
                # 避免请求过于频繁
                time.sleep(0.5)
                
            except Exception as e:
                print(f"  ✗ 获取失败: {e}")
        
        # 保存汇总数据
        if not all_data.empty:
            summary_file = f"{self.data_dir}/所有股票日K数据汇总.xlsx"
            all_data.to_excel(summary_file, index=False)
            print(f"\n✓ 已保存汇总数据到 {summary_file}")
        
        print("=" * 50)
        print(f"批量获取完成: 成功 {success_count}/{len(stock_list)} 只股票")
        
        return all_data
    
    def update_stock_data(self, stock_list, stock_names=None):
        """
        增量更新股票数据（只获取新增部分）
        
        参数:
        stock_list: 股票代码列表
        stock_names: 股票名称列表，可选
        
        返回:
        DataFrame: 新增的数据
        """
        if stock_names is None:
            stock_names = [f"股票_{code}" for code in stock_list]
        
        if len(stock_list) != len(stock_names):
            raise ValueError("股票代码列表和股票名称列表长度不匹配")
        
        today = datetime.now()
        end_date = today.strftime("%Y%m%d")
        
        print(f"开始增量更新 {len(stock_list)} 只股票数据...")
        print(f"更新到日期: {end_date}")
        print("=" * 50)
        
        all_updated_data = pd.DataFrame()
        update_count = 0
        
        for i, stock_code in enumerate(stock_list):
            stock_name = stock_names[i]
            file_name = f"{self.data_dir}/{stock_name}_{stock_code}_日K数据.xlsx"
            
            print(f"[{i+1}/{len(stock_list)}] 检查 {stock_name}({stock_code})")
            
            # 确定开始日期
            start_date = "20200101"  # 默认起始日期
            existing_data = pd.DataFrame()
            
            if os.path.exists(file_name):
                try:
                    existing_data = pd.read_excel(file_name)
                    
                    if not existing_data.empty:
                        # 获取最新日期
                        latest_date = pd.to_datetime(existing_data["日期"].max())
                        
                        # 如果数据已经是最新的，跳过
                        if latest_date.date() >= today.date():
                            print(f"  ✓ 数据已是最新，无需更新")
                            continue
                        
                        # 设置开始日期为最新日期的下一天
                        start_date = (latest_date + timedelta(days=1)).strftime("%Y%m%d")
                        print(f"  → 最新数据日期: {latest_date.strftime('%Y-%m-%d')}")
                        print(f"  → 更新日期范围: {start_date} 到 {end_date}")
                    else:
                        print(f"  → 文件存在但为空，重新获取数据")
                except Exception as e:
                    print(f"  ⚠ 读取现有文件失败: {e}")
            else:
                print(f"  → 文件不存在，首次获取数据")
            
            try:
                # 获取新增数据
                new_data = ak.stock_zh_a_hist(
                    symbol=stock_code, 
                    period="daily", 
                    start_date=start_date, 
                    end_date=end_date,
                    adjust=""
                )
                
                if not new_data.empty:
                    if not existing_data.empty:
                        # 合并新旧数据
                        combined_data = pd.concat([existing_data, new_data], ignore_index=True)
                        # 去重并按日期排序
                        combined_data = combined_data.drop_duplicates(subset=["日期"]).sort_values("日期")
                        combined_data.to_excel(file_name, index=False)
                        print(f"  ✓ 已更新数据，新增 {len(new_data)} 条记录")
                    else:
                        # 直接保存新数据
                        new_data.to_excel(file_name, index=False)
                        print(f"  ✓ 已创建文件，包含 {len(new_data)} 条记录")
                    
                    # 添加到汇总数据
                    new_data["股票代码"] = stock_code
                    new_data["股票名称"] = stock_name
                    all_updated_data = pd.concat([all_updated_data, new_data], ignore_index=True)
                    
                    update_count += 1
                else:
                    print(f"  ○ 没有新数据")
                
                time.sleep(0.5)  # 避免请求过于频繁
                
            except Exception as e:
                print(f"  ✗ 更新失败: {e}")
        
        # 更新汇总文件
        if not all_updated_data.empty:
            summary_file = f"{self.data_dir}/所有股票日K数据汇总.xlsx"
            
            if os.path.exists(summary_file):
                try:
                    existing_summary = pd.read_excel(summary_file)
                    combined_summary = pd.concat([existing_summary, all_updated_data], ignore_index=True)
                    # 去重并排序
                    combined_summary = combined_summary.drop_duplicates(
                        subset=["日期", "股票代码"]).sort_values(["股票代码", "日期"])
                    combined_summary.to_excel(summary_file, index=False)
                    print(f"\n✓ 已更新汇总数据文件")
                except Exception as e:
                    print(f"\n⚠ 更新汇总文件失败: {e}")
            else:
                all_updated_data.to_excel(summary_file, index=False)
                print(f"\n✓ 已创建汇总数据文件")
        
        print("=" * 50)
        print(f"增量更新完成: 更新 {update_count}/{len(stock_list)} 只股票")
        
        return all_updated_data
    
    def get_stock_info(self, stock_code):
        """获取股票基本信息"""
        try:
            # 获取股票基本信息
            info = ak.stock_individual_info_em(symbol=stock_code)
            return info
        except Exception as e:
            print(f"获取股票 {stock_code} 信息失败: {e}")
            return pd.DataFrame()


def main():
    """主程序 - 演示各种功能"""
    print("🔥 股票数据采集器")
    print("=" * 60)
    
    # 创建采集器实例
    collector = StockDataCollector()
    
    # 预定义的股票列表
    stock_codes = ["600519", "601318", "000858", "600036", "000002"]
    stock_names = ["贵州茅台", "中国平安", "五粮液", "招商银行", "万科A"]
    
    while True:
        print("\n请选择功能:")
        print("1. 获取单个股票数据")
        print("2. 批量获取股票数据")
        print("3. 增量更新股票数据")
        print("4. 查看预设股票列表")
        print("5. 自定义股票列表")
        print("0. 退出程序")
        print("-" * 30)
        
        choice = input("请输入选项 (0-5): ").strip()
        
        if choice == "0":
            print("👋 程序已退出")
            break
        
        elif choice == "1":
            print("\n📈 单个股票数据获取")
            stock_code = input("请输入股票代码 (如601318): ").strip()
            stock_name = input("请输入股票名称 (可选，直接回车跳过): ").strip()
            start_date = input("请输入开始日期 (如20230101，直接回车使用默认): ").strip()
            
            if not stock_code:
                print("股票代码不能为空")
                continue
            
            if not stock_name:
                stock_name = None
            
            if not start_date:
                start_date = "20230101"
            
            collector.get_single_stock(stock_code, stock_name, start_date)
        
        elif choice == "2":
            print("\n📊 批量获取股票数据")
            print("使用预设股票列表:")
            for i, (code, name) in enumerate(zip(stock_codes, stock_names)):
                print(f"  {i+1}. {name}({code})")
            
            start_date = input("请输入开始日期 (如20230101，直接回车使用默认): ").strip()
            if not start_date:
                start_date = "20230101"
            
            collector.batch_get_stocks(stock_codes, stock_names, start_date)
        
        elif choice == "3":
            print("\n🔄 增量更新股票数据")
            print("将更新预设股票列表中的所有股票...")
            
            collector.update_stock_data(stock_codes, stock_names)
        
        elif choice == "4":
            print("\n📋 当前预设股票列表:")
            for i, (code, name) in enumerate(zip(stock_codes, stock_names)):
                print(f"  {i+1}. {name}({code})")
        
        elif choice == "5":
            print("\n✏️ 自定义股票列表")
            print("请输入股票代码，用逗号分隔 (如: 600519,601318,000858)")
            codes_input = input("股票代码: ").strip()
            
            if not codes_input:
                print("股票代码不能为空")
                continue
            
            custom_codes = [code.strip() for code in codes_input.split(",")]
            
            print("请输入对应的股票名称，用逗号分隔 (可选)")
            names_input = input("股票名称: ").strip()
            
            if names_input:
                custom_names = [name.strip() for name in names_input.split(",")]
                if len(custom_names) != len(custom_codes):
                    print("股票名称数量与代码数量不匹配，将使用默认名称")
                    custom_names = None
            else:
                custom_names = None
            
            action = input("选择操作 (1-批量获取, 2-增量更新): ").strip()
            
            if action == "1":
                start_date = input("请输入开始日期 (如20230101，直接回车使用默认): ").strip()
                if not start_date:
                    start_date = "20230101"
                collector.batch_get_stocks(custom_codes, custom_names, start_date)
            elif action == "2":
                collector.update_stock_data(custom_codes, custom_names)
            else:
                print("无效选项")
        
        else:
            print("无效选项，请重新选择")


if __name__ == "__main__":
    main() 